In [10]:
import pickle
import gensim
from os import listdir

In [29]:
[_, ptts] = pickle.load(open("/tmp2/GorsachiusMelanolophus/afterProcessing/big/newBlogs_newPTTs_sen.p", "rb"))

In [38]:
for ptt in ptts:
    doc = []
    for line in ptt['content']:
        doc.extend(line)
    docs.append(doc)

In [39]:
taggedDocs = []
for uid, doc in enumerate(docs):
    taggedDoc = gensim.models.doc2vec.TaggedDocument(doc, [uid])
    taggedDocs.append(taggedDoc)

In [40]:
model = gensim.models.doc2vec.Doc2Vec(taggedDocs, size=200, window=8, min_count=5, workers=4)

In [ ]:
def getTrainingData(ptts, docvecs, feature_size):
    X = np.zeros((len(ptts), feature_size))
    y = []
    for doc_i, docvec in enumerate(docvecs):
        if docvec != None:
            X[doc_i] = np.copy(docvec)
            if ptts[doc_i]['isSponsoredPost'] == True:
                y.append(0)
            else:
                y.append(1)
    y = np.asarray(y)
    return X, y

In [ ]:
def main():
    args = parse_args()
    feature_size = 200
    
    path_S='/tmp2/GorsachiusMelanolophus/ptt_posts_new/sponsored/'
    sN = len(listdir(path_S))
    S_X, S_y = getTrainingData(ptts[:sN], model.docvecs[:sN], feature_size)
    notS_X, notS_y = getTrainingData(ptts[sN:], model.docvecs[sN:], feature_size)
    X_train, y_train = np.concatenate((S_X[:int(0.7*len(S_X))], notS_X[:int(0.7*len(notS_X))])), np.concatenate((S_y[:int(0.7*len(S_y))], notS_y[:int(0.7*len(notS_y))]))
    X_valid, y_valid = np.concatenate((S_X[int(0.7*len(S_X)):], notS_X[int(0.7*len(notS_X)):])), np.concatenate((S_y[int(0.7*len(S_y)):], notS_y[int(0.7*len(notS_y)):]))
    
    maxAcc = 0
    for i in range(times):
        if args.model_type == 'svc':
            #clf = SVC(kernel='linear')
            clf = SVC(kernel='rbf')
        elif args.model_type == 'nb':
            clf = GaussianNB()
        elif args.model_type == 'dt':
            clf = DecisionTreeClassifier(random_state=0)
        else:
            raise ValueError('Wrong arg: model_type')

        # balanceData
        X_train_sample, y_train_sample  = balanceData(X_train, y_train)
        # shuffle
        X_train_sample, y_train_sample  = shuffle(X_train_sample, y_train_sample)
        clf.fit(X_train_sample, y_train_sample)
        pred = clf.predict(X_valid)
        acc = np.mean([1 if pred[i] != y_valid[i] else 0 for i in range(len(y_valid))])
        print('Validation acc:', acc)
        if acc > maxAcc:
            maxAcc = acc
    print('maxAcc:', maxAcc)